In [1]:
# References:
# [1] https://www.youtube.com/watch?v=Mut_u40Sqz4
# [2] https://github.com/ARISE-Initiative/robosuite/issues/131

# Dependencies

In [3]:
# !pip install stable-baselines3[extra]
# Robosuite
# gym

# !echo $LD_PRELOAD
# !echo $LD_LIBRARY_PATH


# Required libraries

In [1]:
import os
# openai 
import gym
# Stable baseline imports
from stable_baselines3 import PPO
from stable_baselines3.common.save_util import save_to_zip_file, load_from_zip_file
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
# Robosuite imports
import robosuite as suite
from robosuite.wrappers import GymWrapper
from robosuite.environments.base import register_env
from robosuite import load_controller_config

# Random Action: No Training

In [2]:
# Notice how the environment is wrapped by the wrapper
env = GymWrapper(
    suite.make(
                "TwoArmPegInHole",
                robots=["Panda", "Panda"],  # use Sawyer robot
                use_camera_obs=False,  # do not use pixel observations
                has_offscreen_renderer=False,  # not needed since not using pixel obs
                has_renderer=True,  # make sure we can render to the screen
                reward_shaping=True,  # use dense rewards
                control_freq=20,  # control should happen fast enough so that simulation looks smooth
                horizon = 200,   
                )
            )

n_episodes = 1
for i_episode in range(n_episodes):
    observation = env.reset()
    done = False
    score = 0
    while not done:
        # env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {i_episode} Score: {score}")

env.close()

Creating window glfw
Creating window glfw


/home/damnghost/miniconda3/envs/rl_ultrasound/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Creating window glfw
Cumulative reward: 51.19243374143408
Episode: 0 Score: 51.19243374143408


# Environment Specification

In [3]:
print(f"Initial State observations: \n {env.reset()}")
print(f"Action space: \n{env.action_space}")
print(f"Observation space: \n{env.observation_space}")
env.close()

Creating window glfw
Initial State observations: 
 [ 4.95142844e-03  4.29423525e-02  9.45327754e-01 -5.07854808e-01
 -4.86405216e-01  5.37167311e-01  4.65773271e-01  7.39789776e-03
  1.82895540e-01 -2.59094309e-02  7.16365760e-01  6.95690590e-01
  3.52831775e-02 -3.98722730e-02  1.12595528e-02  6.49378583e-02
  2.77020992e-01  9.99999846e-01  9.76507470e-01  9.99985555e-01
 -8.59645347e-01  9.99888004e-01 -9.77482756e-01  6.92063246e-01
  5.54683879e-04  2.15483554e-01  5.37496485e-03 -5.10891258e-01
  1.49659643e-02  2.11015312e-01  7.21836867e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.80606205e-03 -1.53761606e-01
  1.10050016e+00  7.16365760e-01  6.95690590e-01  3.52831775e-02
 -3.98722730e-02  9.99986919e-01  9.84506822e-01  9.99976065e-01
 -8.81280485e-01  9.99745903e-01 -9.75248031e-01  7.27585804e-01
  5.11482749e-03  1.75346276e-01 -6.91880364e-03 -4.72593597e-01
 -2.25417244e-02  2.21113723e-01  6.860

# Train RL Model

In [5]:
# Save model directories
log_dir = os.path.join('../Training', 'Logs')
print(log_dir)

../Training/Logs


In [ ]:
# def wrap_env(env):
#     wrapped_env = Monitor(env)                          # Needed for extracting eprewmean and eplenmean
#     wrapped_env = DummyVecEnv([lambda : wrapped_env])   # Needed for all environments (e.g. used for mulit-processing)
#     wrapped_env = VecNormalize(wrapped_env)             # Needed for improving training when using MuJoCo envs?
#     return wrapped_env

In [6]:
# env = wrap_env(env)
model = PPO('MlpPolicy', env, n_verbose=1, tensorboard_log=log_dir, )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
model.learn(total_timesteps=400, tb_log_name="TwoArmPegInHole_PPO_TEST")

Creating window glfw
Logging to ../Training/Logs/TwoArmPegInHole_PPO_TEST_1
Cumulative reward: 77.55443670697893
Creating window glfw
Cumulative reward: 59.345476038922406
Creating window glfw
Cumulative reward: 70.1458393912549
Creating window glfw
Cumulative reward: 49.039850974808374
Creating window glfw
Cumulative reward: 56.70263568973142
Creating window glfw
Cumulative reward: 53.891553581972985
Creating window glfw
Cumulative reward: 72.96912767480792
Creating window glfw
Cumulative reward: 66.53560320023722
Creating window glfw
Cumulative reward: 50.349676705899086
Creating window glfw
Cumulative reward: 56.50629471338264
Creating window glfw
Cumulative reward: 57.604742235724956
Creating window glfw
Cumulative reward: 69.56481337716866
Creating window glfw
Cumulative reward: 65.51838074097871
Creating window glfw


KeyboardInterrupt: 

In [ ]:
PPO_path = os.path.join('../Training', 'Saved_Models', 'TwoArmPegInHole_PPO_model')
PPO_path

In [ ]:
model.save(PPO_path)
del model

In [ ]:
ENV_path = os.path.join('../Training', 'Saved_Env/')
env.save("TwoArmPegInHole_VecNorm_Env.pkl")
ENV_path